<a href="https://colab.research.google.com/github/Misharosejoseph/Proanalyst-assignment-/blob/main/ProAnalyst_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT LIBRARIES

In [ ]:
import pandas as pd
import os


STEP 1: LOAD DATASETS (CSV FILES)

In [ ]:
data_path = "brazilian_ecommerce/"

files = [f for f in os.listdir(data_path) if f.endswith(".csv")]
dataframes = {
    f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f))
    for f in files
}

files


['olist_order_payments_dataset.csv',
 'olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_products_dataset.csv',
 'olist_sellers_dataset.csv',
 'olist_orders_dataset.csv',
 'olist_order_items_dataset.csv',
 'product_category_name_translation.csv']

STEP 2: PREPARE BASE TABLES

In [ ]:
orders = dataframes['olist_orders_dataset']
customers = dataframes['olist_customers_dataset']
order_items = dataframes['olist_order_items_dataset']
products = dataframes['olist_products_dataset']
payments = dataframes['olist_order_payments_dataset']


STEP 3: DATA CLEANING (DATES)

In [ ]:
orders['order_purchase_timestamp'] = pd.to_datetime(
    orders['order_purchase_timestamp']
)

orders['order_delivered_customer_date'] = pd.to_datetime(
    orders['order_delivered_customer_date']
)

orders['order_estimated_delivery_date'] = pd.to_datetime(
    orders['order_estimated_delivery_date']
)


STEP 4: CREATE orders_enriched (SQL JOIN REPLACEMENT)

In [ ]:
orders_enriched = orders.merge(customers, on='customer_id', how='left')
orders_enriched = orders_enriched.merge(order_items, on='order_id', how='left')
orders_enriched = orders_enriched.merge(
    products[['product_id', 'product_category_name']],
    on='product_id',
    how='left'
)
orders_enriched = orders_enriched.merge(payments, on='order_id', how='left')

orders_enriched.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,payment_sequential,payment_type,payment_installments,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,1.0,credit_card,1.0,18.12
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,3.0,voucher,1.0,2.00
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,2.0,voucher,1.0,18.59
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,47813,...,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,perfumaria,1.0,boleto,1.0,141.46
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,automotivo,1.0,credit_card,3.0,179.12


A3. BASIC KPIs

A3.1 Overall KPIs

In [ ]:
orders_enriched['order_purchase_date'] = (
    orders_enriched['order_purchase_timestamp'].dt.date
)

total_orders = orders_enriched['order_id'].nunique()
total_revenue = orders_enriched['payment_value'].sum()
avg_order_value = total_revenue / total_orders

date_min = orders_enriched['order_purchase_date'].min()
date_max = orders_enriched['order_purchase_date'].max()

total_orders, total_revenue, avg_order_value, date_min, date_max


(99441,
 np.float64(20470726.659999996),
 np.float64(205.85801289206663),
 datetime.date(2016, 9, 4),
 datetime.date(2018, 10, 17))

A3.2 Yearly Summary

In [ ]:
orders_enriched['year'] = (
    orders_enriched['order_purchase_timestamp'].dt.year
)

yearly_summary = (
    orders_enriched
    .groupby('year')
    .agg(
        total_orders=('order_id', 'nunique'),
        total_revenue=('payment_value', 'sum')
    )
    .reset_index()
)

yearly_summary['avg_order_value'] = (
    yearly_summary['total_revenue'] /
    yearly_summary['total_orders']
)

yearly_summary


,year,total_orders,total_revenue,avg_order_value
0,2016,329,76528.26,232.608693
1,2017,45101,9266612.28,205.463566
2,2018,54011,11127586.12,206.024442


A4. SALES TREND OVER TIME (MONTHLY)

In [ ]:
orders_enriched['year_month'] = (
    orders_enriched['order_purchase_timestamp']
    .dt.to_period('M')
    .astype(str)
)

monthly_sales = (
    orders_enriched
    .groupby('year_month')
    .agg(
        total_orders=('order_id', 'nunique'),
        total_revenue=('payment_value', 'sum')
    )
    .reset_index()
)

monthly_sales['avg_order_value'] = (
    monthly_sales['total_revenue'] /
    monthly_sales['total_orders']
)

monthly_sales.head()



,year_month,total_orders,total_revenue,avg_order_value
0,2016-09,4,388.47,97.117500
1,2016-10,324,76120.17,234.938796
2,2016-12,1,19.62,19.620000
3,2017-01,800,189015.66,236.269575
4,2017-02,1780,349701.93,196.461758


A5. PRODUCT CATEGORY PERFORMANCE

In [ ]:
category_performance = (
    orders_enriched
    .groupby('product_category_name')
    .agg(
        total_orders=('order_id', 'nunique'),
        total_items=('order_id', 'count'),
        total_revenue=('payment_value', 'sum'),
        avg_price=('price', 'mean')
    )
    .reset_index()
)

category_performance = category_performance.sort_values(
    by='total_revenue',
    ascending=False
).head(10)

category_performance


,product_category_name,total_orders,total_items,total_revenue,avg_price
13,cama_mesa_banho,9417,11823,1712553.67,92.408950
11,beleza_saude,8836,9975,1657373.12,130.074263
44,informatica_acessorios,6689,8082,1585330.45,116.589652
54,moveis_decoracao,6449,8744,1430176.39,87.499301
66,relogios_presentes,5624,6201,1429216.68,202.087292
32,esporte_lazer,7720,8945,1392127.56,114.476952
72,utilidades_domesticas,5884,7355,1094758.13,90.630455
8,automotivo,3897,4379,852294.33,140.843231
40,ferramentas_jardim,3518,4574,838280.75,113.296358
26,cool_stuff,3632,3987,779698.00,166.117254


A6. STATE-LEVEL PERFORMANCE

In [ ]:
state_performance = (
    orders_enriched
    .groupby('customer_state')
    .agg(
        unique_customers=('customer_id', 'nunique'),
        total_orders=('order_id', 'nunique'),
        total_revenue=('payment_value', 'sum')
    )
    .reset_index()
)

state_performance['avg_order_value'] = (
    state_performance['total_revenue'] /
    state_performance['total_orders']
)

state_performance = state_performance.sort_values(
    by='total_revenue',
    ascending=False
)

state_performance.head()


,customer_state,unique_customers,total_orders,total_revenue,avg_order_value
25,SP,41746,41746,7673188.55,183.806558
18,RJ,12852,12852,2783724.26,216.598526
10,MG,11635,11635,2341861.47,201.277307
22,RS,5466,5466,1152019.17,210.760917
17,PR,5045,5045,1074614.19,213.005786


A7. DELIVERY PERFORMANCE

A7.1 Create Delivery Columns

In [ ]:
orders_enriched['delivery_time_days'] = (
    orders_enriched['order_delivered_customer_date'] -
    orders_enriched['order_purchase_timestamp']
).dt.days

orders_enriched['estimated_delivery_days'] = (
    orders_enriched['order_estimated_delivery_date'] -
    orders_enriched['order_purchase_timestamp']
).dt.days


A7.2 Delivery Summary by State

In [ ]:
delivery_summary_by_state = (
    orders_enriched
    .groupby('customer_state')
    .agg(
        avg_delivery_time_days=('delivery_time_days', 'mean'),
        avg_estimated_delivery_days=('estimated_delivery_days', 'mean')
    )
    .reset_index()
)

delivery_summary_by_state.head()


,customer_state,avg_delivery_time_days,avg_estimated_delivery_days
0,AC,20.170213,40.747368
1,AL,24.138322,32.165217
2,AM,26.100592,45.180233
3,AP,27.662651,45.654762
4,BA,18.721628,29.091378


A8. EXPORT CSV FILES (FINAL STEP)

In [ ]:
orders_enriched.to_csv("orders_enriched.csv", index=False)
monthly_sales.to_csv("monthly_sales.csv", index=False)
category_performance.to_csv("category_performance.csv", index=False)
state_performance.to_csv("state_performance.csv", index=False)
delivery_summary_by_state.to_csv("delivery_summary_by_state.csv", index=False)

print("✅ All Part A CSV files exported successfully")


✅ All Part A CSV files exported successfully
